#Import Libraries. Request-function

In [ ]:
import csv
import datetime as dt
import json
import os
import statistics
import time


import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from google.colab import files


# ensure tables show all columns
pd.options.display.max_columns = 100

In [ ]:
ukr_developer = ['4A Games', 'Beatshapers', 'Best Way', 'Frogwares','Game-Labs',
                                 'GSC Game World','Room 8 Studio','VG Entertainment']


##Parse Ukrainian Games from Wikipedia

In [ ]:
url = f'https://en.wikipedia.org/wiki/Category:Video_games_developed_in_Ukraine'
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    games_div = soup.find('div', {'class': 'mw-category-columns'})

    game_names = [link.text for link in games_div.find_all('a')]

game_list = []
for game in game_names:
    game = game.split(' (')[0]
    game_list.append(game)
print(game_list)


['80 Days', 'Achtung Panzer: Kharkov 1943', 'The Age of Decadence', 'Alexander', 'American Conquest', 'Atom RPG', 'Big Rigs: Over the Road Racing', 'Boiling Point: Road to Hell', 'Bubble Shooter', 'Buku Sudoku', 'Carnivores', 'Carnivores 2', 'Carnivores: Dinosaur Hunter', 'Carnivores: Ice Age', 'Chasm: The Rift', 'Codename: Outbreak', 'Collapse', 'Contre Jour', 'Cossacks', 'Cossacks 3', 'Cossacks II: Napoleonic Wars', 'Cossacks: European Wars', 'Cradle', 'CrimeCraft', 'Cryostasis: Sleep of Reason', 'Cyto', 'Darkened Skye', 'Dracula: Origin', 'Faces of War', 'Far Cry New Dawn', 'Football, Tactics & Glory', 'Game-Labs', 'Glass Tower', 'Heroes of Annihilated Empires', 'Hover Ace', 'Journey to the Center of the Earth', 'Magrunner: Dark Pulse', 'Men of War', 'Metro 2033', 'Metro Exodus', 'Metro: Last Light', 'Mount & Blade: With Fire & Sword', 'Mystery Case Files', 'Neuro Hunter', 'No One Lives Under the Lighthouse', 'Nuclear Union', 'Party Hard', 'Pinball HD', 'The Precursors', 'Robocalyps

#Access to Steamspy api

### Get only Ukrainian Games statistics
### Get Games name and its SteamID

In [ ]:
def get_data_from_url(url):
    response = requests.get(url)

    if response.status_code == 200:
        return response.json().get('applist', {}).get('apps', [])
    else:
        print(f"Failed. Status code: {response.status_code}")
        return []

def get_app_ids_for_games(required_games, url):
    url_data = get_data_from_url(url)

    app_ids = {'Game': [], 'App ID': []}

    for game in url_data:
        app_name = game.get('name', '')
        if app_name in required_games:
            app_ids['Game'].append(app_name)
            app_ids['App ID'].append(game.get('appid', ''))

    return pd.DataFrame(app_ids)

# List of games for which you want to get app_ids
required_games = game_list

# URL to fetch the data from
url = 'https://api.steampowered.com/ISteamApps/GetAppList/v2/'

# Get data from the URL and create a DataFrame
df = get_app_ids_for_games(required_games, url)

# Print the DataFrame
print(df)

                                     Game   App ID
0                                Collapse  1511000
1                       The Serpent Rogue  1453790
2                         Chasm: The Rift  2061230
3                        The Sinking City  1151760
4             Sherlock Holmes Chapter One  1137300
5                    This Land Is My Land  1069640
6                        Far Cry New Dawn   939960
7                      The Sinking City 2  2825860
8                                 80 Days   381780
9               Football, Tactics & Glory   375530
10                                 Cradle   361550
11                   The Age of Decadence   230070
12                             Party Hard   356570
13  Sherlock Holmes: The Devil's Daughter   350640
14                             Cossacks 3   333420
15                             Metro 2033    43129
16        S.T.A.L.K.E.R.: Call of Pripyat    41700
17                    Stalin vs. Martians    25870
18                    Stalin vs

In [ ]:

def get_game_info(app_id):
    api_url = f'https://steamspy.com/api.php?request=appdetails&appid={app_id}'
    response = requests.get(api_url)

    if response.status_code == 200:
        game_info = response.json()
        return game_info
    else:
        print(f"Failed to parse data for app ID {app_id}")
        return None

# List of app IDs
app_ids = df["App ID"]  # Replace with your own list of app IDs

# Create a dataset
dataset = []

# Fetch information for each app ID
for app_id in app_ids:
    game_info = get_game_info(app_id)

    if game_info:
        dataset.append({
            'appid': game_info.get('appid'),
            'name': game_info.get('name'),
            'developer': game_info.get('developer'),
            'publisher': game_info.get('publisher'),
            'score_rank': game_info.get('score_rank'),
            'owners': game_info.get('owners'),
            'average_forever': game_info.get('average_forever'),
            'average_2weeks': game_info.get('average_2weeks'),
            'median_forever': game_info.get('median_forever'),
            'median_2weeks': game_info.get('median_2weeks'),
            'ccu': game_info.get('ccu'),
            'price': game_info.get('price'),
            'tags': game_info.get('tags'),
            'languages': game_info.get('languages'),
            'genre': game_info.get('genre')
        })

# Print or further process the dataset

df = pd.DataFrame(dataset)
#for entry in dataset:
   # print(entry)


In [ ]:
df.head()

,appid,name,developer,publisher,score_rank,owners,average_forever,average_2weeks,median_forever,median_2weeks,ccu,price,tags,languages,genre
0,1511000,Collapse,MoonKang,MoonKang,,"0 .. 20,000",0,0,0,0,0,399,"{'Casual': 64, 'Time Management': 36, '3D': 35...","English, Simplified Chinese","Casual, Indie, Strategy"
1,1453790,The Serpent Rogue,Sengi Games,Team17,,"50,000 .. 100,000",0,0,0,0,3,1999,"{'Crafting': 497, 'Adventure': 486, 'Action-Ad...","English, Russian, Ukrainian, French, German, S...","Action, Adventure, Indie, RPG"
2,2061230,Chasm: The Rift,"Action Forms, General Arcade",SNEG,,"0 .. 20,000",0,0,0,0,0,1999,"{'FPS': 102, 'Retro': 94, 'Shooter': 87, 'Old ...","English, French, Italian, German, Spanish - Sp...",Action
3,1151760,The Sinking City,Frogwares,Frogwares,,"50,000 .. 100,000",0,0,0,0,4,0,"{'Action-Adventure': 75, 'Adventure': 74, 'Ope...","English, French, Italian, German, Spanish - Sp...","Action, Adventure"
4,1137300,Sherlock Holmes Chapter One,Frogwares,Frogwares,,"200,000 .. 500,000",762,0,1131,0,51,4499,"{'Detective': 345, 'Open World': 312, 'Story R...","English, French, Italian, German, Spanish - Sp...","Action, Adventure"


In [ ]:
df.size

675

###Save the dataset to csv file

In [ ]:
df.to_csv('steam_ukr_games.csv', index=False)
files.download('steam_ukr_games.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>